In [5]:
import cv2
import os


# 打开视频文件
video_path = 'TS-20241123222043114.avi'
cap = cv2.VideoCapture(video_path)

# 创建输出目录
output_dir = f'{video_path}_frames'
os.makedirs(output_dir, exist_ok=True)


# 获取视频帧率
original_fps = cap.get(cv2.CAP_PROP_FPS)
print(f"原始视频帧率: {original_fps} FPS")

# 自定义帧率 (例如，提取 10 FPS)
custom_fps = 6
frame_interval = int(original_fps / custom_fps)

frame_id = 0
saved_frame_id = 0

# 逐帧读取
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 按照帧间隔保存
    if frame_id % frame_interval == 0:
        saved_frame_id += 1
        output_path = os.path.join(output_dir, f"{video_path}_{saved_frame_id:04d}.tiff")
        cv2.imwrite(output_path, frame)

    frame_id += 1

cap.release()
print("自定义帧率的帧提取完成！")



原始视频帧率: 28.8 FPS
自定义帧率的帧提取完成！


In [8]:
import os
import json
import cv2

# 定义路径
labelme_dir = "./TS-20241123222043114.avi_frames/annotations"  # Labelme JSON 文件目录
output_dir = "./TS-20241123222043114.avi_frames/yolo_TS-20241123220321351"  # 转换后的 YOLO 标注文件保存目录
os.makedirs(output_dir, exist_ok=True)

# 类别字典（支持多个类别）
classes = {"head": 0, "tail": 1}  # 自定义类别及其对应的 ID

# 遍历所有 JSON 文件
for filename in os.listdir(labelme_dir):
    if filename.endswith(".json"):
        json_path = os.path.join(labelme_dir, filename)

        # 读取 JSON 文件
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        # 获取图片宽高
        image_path = os.path.join(labelme_dir, data["imagePath"])
        image = cv2.imread(image_path)
        if image is None:
            print(f"无法读取图片: {image_path}")
            continue
        height, width, _ = image.shape

        # YOLO 格式标注结果
        yolo_labels = []

        # 处理每个标注
        for shape in data["shapes"]:
            label = shape["label"]
            if label in classes:  # 确保类别在类别字典中
                points = shape["points"]
                x_min = min([p[0] for p in points])
                y_min = min([p[1] for p in points])
                x_max = max([p[0] for p in points])
                y_max = max([p[1] for p in points])

                # 转换为 YOLO 格式
                x_center = (x_min + x_max) / 2 / width
                y_center = (y_min + y_max) / 2 / height
                bbox_width = (x_max - x_min) / width
                bbox_height = (y_max - y_min) / height

                class_id = classes[label]
                yolo_labels.append(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}")

        # 保存为 YOLO 格式文件
        yolo_filename = os.path.join(output_dir, filename.replace(".json", ".txt"))
        with open(yolo_filename, "w") as f:
            f.write("\n".join(yolo_labels))

        print(f"转换完成：{filename} -> {yolo_filename}")

print("所有标注文件已转换为 YOLO 格式！")


转换完成：TS-20241123222043114.avi_0001.json -> ./TS-20241123222043114.avi_frames/yolo_TS-20241123220321351\TS-20241123222043114.avi_0001.txt
转换完成：TS-20241123222043114.avi_0009.json -> ./TS-20241123222043114.avi_frames/yolo_TS-20241123220321351\TS-20241123222043114.avi_0009.txt
转换完成：TS-20241123222043114.avi_0014.json -> ./TS-20241123222043114.avi_frames/yolo_TS-20241123220321351\TS-20241123222043114.avi_0014.txt
转换完成：TS-20241123222043114.avi_0022.json -> ./TS-20241123222043114.avi_frames/yolo_TS-20241123220321351\TS-20241123222043114.avi_0022.txt
转换完成：TS-20241123222043114.avi_0032.json -> ./TS-20241123222043114.avi_frames/yolo_TS-20241123220321351\TS-20241123222043114.avi_0032.txt
转换完成：TS-20241123222043114.avi_0037.json -> ./TS-20241123222043114.avi_frames/yolo_TS-20241123220321351\TS-20241123222043114.avi_0037.txt
转换完成：TS-20241123222043114.avi_0120.json -> ./TS-20241123222043114.avi_frames/yolo_TS-20241123220321351\TS-20241123222043114.avi_0120.txt
转换完成：TS-20241123222043114.avi_0170.json -

In [9]:
import cv2
import os

# 图片和标注目录
image_dir = "./TS-20241123222043114.avi_frames"  # 图片目录
label_dir = "./TS-20241123222043114.avi_frames/yolo_TS-20241123220321351"  # YOLO 标注目录

for label_file in os.listdir(label_dir):
    if label_file.endswith(".txt"):
        # 加载图片
        image_path = os.path.join(image_dir, label_file.replace(".txt", ".tiff"))
        image = cv2.imread(image_path)

        # 加载标注
        with open(os.path.join(label_dir, label_file), "r") as f:
            for line in f.readlines():
                class_id, x_center, y_center, width, height = map(float, line.strip().split())
                img_h, img_w, _ = image.shape
                x1 = int((x_center - width / 2) * img_w)
                y1 = int((y_center - height / 2) * img_h)
                x2 = int((x_center + width / 2) * img_w)
                y2 = int((y_center + height / 2) * img_h)

                # 绘制矩形框
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(image, f"Class {int(class_id)}", (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # 显示图片
        cv2.imshow("Annotated Image", image)
        cv2.waitKey(0)

cv2.destroyAllWindows()


In [10]:
# 生成类别文件
with open(os.path.join(output_dir, "classes.txt"), "w") as f:
    for class_name in classes:
        f.write(f"{class_name}\n")
print("类别文件 classes.txt 已生成！")


类别文件 classes.txt 已生成！
